In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import os
import time
from typing import List
from pydantic import BaseModel
import openai
from tqdm import tqdm

# Initialize OpenAI client
openai.api_key = os.getenv('OPENAI_API_KEY')

def duckduckgo_search(query: str, max_results: int = 7) -> List[str]:
    """
    Perform a DuckDuckGo search and return a list of result URLs.

    Args:
        query (str): The search query.
        max_results (int): Maximum number of result URLs to return.

    Returns:
        List[str]: A list of URLs from the search results.
    """
    search_url = "https://html.duckduckgo.com/html/"
    params = {
        'q': query
    }
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    try:
        response = requests.post(search_url, data=params, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Error during DuckDuckGo search for query '{query}': {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for result in soup.find_all('a', {'class': 'result__a'}, href=True):
        links.append(result['href'])
        if len(links) >= max_results:
            break
    return links

def extract_text(url: str, max_chars: int = 10000) -> str:
    """
    Extract text content from a URL, limiting to a maximum number of characters.

    Args:
        url (str): The URL to extract text from.
        max_chars (int): Maximum number of characters to extract.

    Returns:
        str: The extracted and cleaned text.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching URL '{url}': {e}")
        return ""

    soup = BeautifulSoup(response.text, 'html.parser')

    # Remove scripts and styles
    for script in soup(["script", "style"]):
        script.decompose()

    text = soup.get_text(separator=' ', strip=True)
    text = re.sub(r'\s+', ' ', text)

    # Limit the text to the first max_chars characters
    return text[:max_chars]

def get_search_text(query: str, max_results: int = 7, max_chars: int = 10000) -> (str, List[str]):
    """
    Perform a search, extract text from result URLs, and combine the text.

    Args:
        query (str): The search query.
        max_results (int): Number of search result URLs to process.
        max_chars (int): Maximum number of characters to extract per URL.

    Returns:
        Tuple[str, List[str]]: Combined extracted text and list of source URLs.
    """
    urls = duckduckgo_search(query, max_results)
    print(f"Found {len(urls)} URLs for query '{query}'.")
    texts = []
    for url in urls:
        text = extract_text(url, max_chars)
        if text:
            texts.append(text)
        time.sleep(1)  # Respectful delay between requests
    combined_text = "\n\n".join(texts)
    return combined_text, urls


class GPTResponse(BaseModel):
    prof_summary: str
    early_career: str
    mid_career: str
    late_career: str
    key_achievements: str
    motivation: str
    vision_mission: str
    personal_input: str
    sources: List[str]



def generate_structured_output(person_name: str, location: str, university: str, combined_text: str, sources: List[str]) -> GPTResponse:

    prompt = f"""
    Based on the following information, fill out the professional summary, early career, mid career, late career, key achievements, motivation, vision/mission, personal input, and sources for {person_name}.

    Location: {location}
    University: {university}

    Extracted Information:
    {combined_text}

    Requirements:
    - prof_summary: A concise professional summary.
    - early_career: Description of early career stages.
    - mid_career: Description of mid-career stages.
    - late_career: Description of late career stages.
    - key_achievements: Highlight key achievements.
    - motivation: What motivated the person in their career.
    - vision_mission: The person's vision or mission.
    - personal_input: Personal insights or quotes.
    - sources: List the provided source URLs.

    Please provide the output in the following JSON format:

    {{
      "fields": {{
        "prof_summary": "",
        "early_career": "",
        "mid_career": "",
        "late_career": "",
        "key_achievements": "",
        "motivation": "",
        "vision_mission": "",
        "personal_input": "",
        "sources": []
      }}
    }}
    """

    response = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an assistant that structures professional profiles based on provided information."},
            {"role": "user", "content": prompt}
        ],
        response_format=GPTResponse,
    )
    return response.choices[0].message.parsed


# Load the CSV file
csv_file_path = 'empty.csv'  # Replace with your actual CSV file path
df = pd.read_csv(csv_file_path)

# Define the fields to be filled
fields_to_fill = ['prof summary', 'early career', 'mid career', 'late career',
                 'key achievements', 'motivation', 'vision/mission', 'personal input', 'sources']

# Check and create missing columns
for field in fields_to_fill:
    if field not in df.columns:
        df[field] = ""

# # Display the first few rows to verify
# df.head()

def process_person(row):
    person_name = row['founder']
    location = row['location']
    university = row['UG school']
    
    query = f"{person_name} {location} {university}"
    print(f"Processing: {person_name}")
    
    combined_text, sources = get_search_text(query)
    
    if not combined_text:
        print(f"No text extracted for {person_name}. Skipping.")
        return row
    
    structured_output = generate_structured_output(person_name, location, university, combined_text, sources)
    
    # Update the DataFrame row with the structured output
    row['prof summary'] = structured_output.prof_summary
    row['early career'] = structured_output.early_career
    row['mid career'] = structured_output.mid_career
    row['late career'] = structured_output.late_career
    row['key achievements'] = structured_output.key_achievements
    row['motivation'] = structured_output.motivation
    row['vision/mission'] = structured_output.vision_mission
    row['personal input'] = structured_output.personal_input
    row['sources'] = ", ".join(structured_output.sources)
    
    # Respectful delay to avoid hitting rate limits
    time.sleep(2)
    
    return row


# Initialize tqdm for progress visualization
total_rows = df.shape[0]
with tqdm(total=total_rows) as pbar:
    for index, row in df.iterrows():
        if pd.isna(row['prof summary']) or row['prof summary'] == "":
            try:
                df.loc[index] = process_person(row)
            except Exception as e:
                print(f"Error processing row {index} for {row['founder']}: {e}")
        else:
            print(f"Skipping {row['founder']} as 'prof summary' already exists.")
        
        # Update progress bar
        pbar.update(1)
        
        # Optional: Save progress periodically every 10 rows
        if index % 10 == 0 and index != 0:
            df.to_csv('updated_empty.csv', index=False)
            print(f"Progress saved at row {index}.")


# Save the updated DataFrame to a new CSV file
updated_csv_file_path = 'updated_empty.csv'  # You can change the file name as needed
df.to_csv(updated_csv_file_path, index=False)
print(f"Updated data saved to {updated_csv_file_path}.")


  0%|          | 0/160 [00:00<?, ?it/s]

Processing: Kunal Agarwal
Found 7 URLs for query 'Kunal Agarwal Palo Alto, California, United States Valparaiso University'.
Error fetching URL 'https://www.crunchbase.com/person/kunal-agarwal-2': 403 Client Error: Forbidden for url: https://www.crunchbase.com/person/kunal-agarwal-2
Error fetching URL 'https://www.intelligence360.news/palo-alto-california-based-unravel-data-is-raising-50000000-00-in-new-equity-investment/': 406 Client Error: Not Acceptable for url: https://www.intelligence360.news/palo-alto-california-based-unravel-data-is-raising-50000000-00-in-new-equity-investment/


/var/folders/5m/w0rlpsxd7yx_97nl1gx0dfzm0000gn/T/ipykernel_79343/775319625.py:223: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Kunal Agarwal is the CEO and co-founder of Unravel Data, a leader in DataOps observability platforms aimed at optimizing performance and cost for modern data teams. With extensive experience in both entrepreneurship and corporate leadership, he has a track record of helping organizations leverage data for actionable insights.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index] = process_person(row)
/var/folders/5m/w0rlpsxd7yx_97nl1gx0dfzm0000gn/T/ipykernel_79343/775319625.py:223: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Kunal started his career leading sales and implementation of Oracle products for several Fortune 100 companies. He playe

Processing: David Aktary
Found 7 URLs for query 'David Aktary Austin, Texas, United States University of Michigan - Dearborn'.
Error fetching URL 'https://www.crunchbase.com/person/david-aktary': 403 Client Error: Forbidden for url: https://www.crunchbase.com/person/david-aktary


  1%|▏         | 2/160 [00:46<58:42, 22.29s/it]  

...

Processing: Sean Wilson
Found 7 URLs for query 'Sean Wilson Raleigh-Durham-Chapel Hill Area Wheaton College in Illinois'.


 99%|█████████▉| 159/160 [52:10<00:19, 19.53s/it]

Processing: Derrick Xiong
Found 7 URLs for query 'Derrick Xiong Los Angeles Metropolitan Area NTU Singapore'.
Error fetching URL 'https://www.crunchbase.com/person/derrick-xiong': 403 Client Error: Forbidden for url: https://www.crunchbase.com/person/derrick-xiong


100%|██████████| 160/160 [52:29<00:00, 19.68s/it]

Updated data saved to updated_empty.csv.
